In [ ]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=20, facecolor='white')
plt.rcParams['figure.figsize']=(10,8)

In [ ]:
adata = sc.read_h5ad("NEWNC_a.new.h5ad")

In [ ]:
adata.obs

In [ ]:
adata
adata.shape

In [ ]:
adata.obs['Stage']=adata.obs['Stage'].astype(str)   ####same in CellType
adata.obs['Stage']

In [ ]:
adata.var["gene_id"] = adata.var.index.values
adata.var

In [ ]:
sc.pl.violin(adata, 'nCount_RNA', groupby='CellType', size=2, log=True, cut=0)

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

In [ ]:
adata.layers["counts"] = adata.X.copy()

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
adata.raw = adata
adata

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', n_top_genes=4000)
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))
sc.pl.highly_variable_genes(adata)

In [ ]:
adata_hvg = adata.copy()
adata_hvg = adata_hvg[:, adata.var['highly_variable']]
adata_hvg
adata_hvg.var['highly_variable']

In [ ]:
adata=adata_backup.copy()
adata_backup = adata.copy()

In [ ]:
# Calculate the visualizations
sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')
sc.pp.neighbors(adata) ######need to personalize
sc.pl.pca_variance_ratio(adata, log=True)#####personalize
sc.tl.tsne(adata, n_jobs=12) #Note n_jobs works for MulticoreTSNE, but not regular implementation)
sc.tl.umap(adata)
sc.tl.diffmap(adata)
sc.tl.draw_graph(adata

In [ ]:
sc.pl.pca_scatter(adata, color='nCount_RNA')
sc.pl.tsne(adata, color='nCount_RNA')
sc.pl.umap(adata, color='nCount_RNA')
sc.pl.draw_graph(adata, color='nCount_RNA')
sc.pl.diffmap(adata, color='Stage', components=['1,2','1,4','1,5','2,3','2,4'])

In [ ]:
#####cellcycle score
cc_genes = pd.read_table('cc_genes_file.txt', delimiter='\t')
cc_genes
s_genes = cc_genes['S'].dropna()
g2m_genes = cc_genes['G2.M'].dropna()
s_genes_mm = [gene.lower().capitalize() for gene in s_genes]
g2m_genes_mm = [gene.lower().capitalize() for gene in g2m_genes]
s_genes_mm_ens = adata.var_names[np.in1d(adata.var_names, s_genes_mm)]
g2m_genes_mm_ens = adata.var_names[np.in1d(adata.var_names, g2m_genes_mm)]
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes_mm_ens, g2m_genes=g2m_genes_mm_ens)

In [ ]:
sc.pl.umap(adata, color=['S_score', 'G2M_score'], use_raw=False)
sc.pl.umap(adata, color='phase', use_raw=False)

In [ ]:
###adata_cellcycleregressfor cell cycle regressout
adata_cellcycleregress = adata.copy()

In [ ]:
adata_cellcycleregress.obs["cell_cycle_diff"] = adata_cellcycleregress.obs["S_score"] - adata_cellcycleregress.obs["G2M_score"]
sc.pp.regress_out(adata_cellcycleregress, ['cell_cycle_diff'])

In [ ]:
adata_cellcycleregress

In [ ]:
sc.pp.pca(adata_cellcycleregress, n_comps=50, use_highly_variable=True, svd_solver='arpack')
sc.pp.neighbors(adata_cellcycleregress)
sc.tl.umap(adata_cellcycleregress)
sc.tl.diffmap(adata_cellcycleregress)
sc.tl.draw_graph(adata_cellcycleregress)

In [ ]:
sc.pl.umap(adata_cellcycleregress, color=['S_score', 'G2M_score'], use_raw=False)
sc.pl.umap(adata_cellcycleregress, color='phase', use_raw=False)

In [ ]:
sc.pl.umap(adata_cellcycleregress, color='CellType')
sc.pl.umap(adata_cellcycleregress, color='Stage')

In [ ]:
sc.pl.umap(adata_cellcycleregress, color='Stage')
sc.pl.umap(adata, color='Stage')

In [ ]:
sc.pl.umap(adata_cellcycleregress, color='CellType')
sc.pl.umap(adata, color='CellType')

In [ ]:
####clustering
sc.tl.louvain(adata, key_added='louvain_r1')
sc.tl.louvain(adata, resolution=0.5, key_added='louvain_r0.5', random_state=10)
sc.tl.louvain(adata, resolution=0.6, key_added='louvain_r0.6', random_state=10)
sc.tl.louvain(adata, resolution=0.8, key_added='louvain_r0.8', random_state=10)
sc.pl.umap(adata, color=['louvain_r1', 'louvain_r0.5','louvain_r0.6','louvain_r0.8'])

In [ ]:
sc.pl.umap(adata, color=['CellType','louvain_r0.8','Stage'],legend_loc='on data')

In [ ]:
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color='Pax7', use_raw=False, color_map=mymap)
sc.pl.umap(adata, color='Alx3', use_raw=False, color_map=mymap)

In [ ]:
#######paga
sc.tl.paga(adata, groups='louvain_r0.8')
sc.pl.paga_compare(adata)
sc.pl.paga(adata)

In [ ]:
sc.pl.paga_compare(adata, basis='umap')

In [ ]:
adata_paga_test = adata.copy()

In [ ]:
sc.tl.paga(adata_paga_test, groups='CellType')
sc.pl.paga_compare(adata_paga_test)
sc.pl.paga(adata_paga_test)